# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [2]:
gdf = cudf.read_csv('./data/week2.csv')

gdf.head()

,lat,long,infected
0,54.522510,-1.571896,0.0
1,54.554030,-1.524968,0.0
2,54.552486,-1.435203,0.0
3,54.537189,-1.566215,0.0
4,54.528212,-1.588462,0.0


## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [3]:
hospitals = cudf.read_csv('./data/hospitals.csv')
clinics = cudf.read_csv('./data/clinics.csv')

all_med = hospitals.append(clinics)

Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [4]:
# Get All rows that are not-null in Latitude and Longitude
filter = cp.logical_and(all_med["Latitude"].notna(), all_med["Longitude"].notna())
all_med = all_med.loc[filter].copy()
all_med = all_med.reset_index()

all_med = all_med.drop(columns=["index"])

all_med

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20296,10617742,NBG7L,Clinic,UNKNOWN,<NA>,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,...,<NA>,BD12 9NG,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20297,10617743,NFH02,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,...,<NA>,BS40 5EL,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20298,10617744,NFH03,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,...,<NA>,BS26 2BJ,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20299,10617746,NEMAM,Clinic,UNKNOWN,<NA>,Visible,False,Lawton House Surgery,Bromley Road,<NA>,...,<NA>,CW12 1QG,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [6]:
all_med["northing"], all_med["easting"] = latlong2osgbgrid_cupy(cp.asarray(all_med["Latitude"]), cp.asarray(all_med["Longitude"]))

all_med[["northing", "easting"]]

,northing,easting
0,165810.473974,510917.517174
1,158381.343420,500604.836652
2,171305.775859,341119.365090
3,395944.561405,383703.600293
4,244071.710013,534945.182860
...,...,...
20296,426887.040596,415302.519662
20297,160625.127012,345341.262026
20298,154503.665383,342794.591249
20299,362932.494686,386243.408863


## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [7]:
knn = cuml.NearestNeighbors(n_neighbors = 1)
knn.fit(all_med[["northing", "easting"]])

NearestNeighbors(n_neighbors=1, verbose=4, handle=<cuml.common.handle.Handle object at 0x7fecf00da290>, algorithm='brute', metric='euclidean', p=2, metric_params=None, output_type='cudf')

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [8]:
infected_gdf = gdf.loc[gdf["infected"] == 1].reset_index().drop(columns=["index"])
infected_gdf.head(5)

,lat,long,infected
0,53.715826,-2.430079,1.0
1,53.664881,-2.425673,1.0
2,53.696765,-2.488940,1.0
3,53.696966,-2.488897,1.0
4,53.727804,-2.392959,1.0


Create `northing` and `easting` values for `infected_gdf`.

In [9]:
infected_gdf["northing"], infected_gdf["easting"] = latlong2osgbgrid_cupy(cp.asarray(infected_gdf["lat"]), cp.asarray(infected_gdf["long"]))

all_med[["northing", "easting"]]

,northing,easting
0,165810.473974,510917.517174
1,158381.343420,500604.836652
2,171305.775859,341119.365090
3,395944.561405,383703.600293
4,244071.710013,534945.182860
...,...,...
20296,426887.040596,415302.519662
20297,160625.127012,345341.262026
20298,154503.665383,342794.591249
20299,362932.494686,386243.408863


Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [10]:
distances, indices = knn.kneighbors(infected_gdf[["northing", "easting"]], 1)

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [11]:
indices.head()

0    18316
1    12816
2     4489
3     4489
4     4962
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [12]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat             53.715826
long            -2.430079
infected         1.000000
northing    424489.783814
easting     371619.678741
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [13]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax,northing,easting
1234,17942,NAA06,Clinic,UNKNOWN,<NA>,Visible,False,Manor Farm Health Centre,Manor Farm Road,<NA>,...,53.404556,-2.831035,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,390099.13988,344757.527843


<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

Please proceed to [the next notebook](3-04_identify_risk_factors.ipynb).